## get table mappings 

In [42]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# Scopes for Sheets + Drive
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Load credentials
creds = Credentials.from_service_account_file(
    "../uw_backend/secrets/service_account_key.json",
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(creds)

# https://docs.google.com/spreadsheets/d/1OikBudPoWjfOpXZXayslZAUZY7O3eM_k7UD-aGkPm9k/edit?gid=1360521348#gid=1360521348
# Open the spreadsheet and worksheet
spreadsheet = client.open_by_key("1OikBudPoWjfOpXZXayslZAUZY7O3eM_k7UD-aGkPm9k")
worksheet = spreadsheet.worksheet("Model Variable Mapping")

# ✅ Get all values including formulas (not evaluated values)
raw_data = worksheet.get_all_values(value_render_option='FORMULA')

# ✅ Convert to DataFrame (first row is headers)
df = pd.DataFrame(raw_data[1:], columns=raw_data[0])

# Preview
df

,section,field_key,location,start_month_location,end_month_location
0,General Property Assumptions,Asking Price,=Assumptions!O3,,
1,General Property Assumptions,Acquisition Price,=Assumptions!G20,,
2,General Property Assumptions,Model Start Date,=Assumptions!F2,,
3,General Property Assumptions,Closing on Property,=Assumptions!G21,,
4,General Property Assumptions,Gross Square Feet,=Assumptions!G22,,
5,General Property Assumptions,Share of Equity from Sponsor,=Assumptions!K3,,
6,Leasing Assumptions,Rehab time,=Assumptions!G51,,
7,Leasing Assumptions,Lease-up Time,=Assumptions!G52,,
8,Leasing Assumptions,Bad Debt,=Assumptions!G54,,
9,Leasing Assumptions,Vacancy,=Assumptions!G55,,


In [51]:
retail_income = [
    {
        "annual_bumps": 1,
        "lease_start_month": 2,
        "recovery_start_month": 0,
        "rent_per_square_foot_per_year": 10,
        "rent_start_month": 4,
        "square_feet": 2000,
        "suite": "asdf",
        "tenant_name": "asdf"
    },
    {
        "annual_bumps": 1,
        "lease_start_month": 1,
        "recovery_start_month": 0,
        "rent_per_square_foot_per_year": 10,
        "rent_start_month": 1,
        "square_feet": 2311,
        "suite": "eee",
        "tenant_name": "eee"
    }
]


growth_rates = [
    {
        "name": "Fair Market",
        "value": 2.5,
        "type": "rental"
    },
    {
        "name": "Rent Control",
        "value": 1,
        "type": "rental"
    },
    {
        "name": "Rent Stabilized",
        "value": 2,
        "type": "rental"
    },
    {
        "name": "Amenity Inflation",
        "value": 2,
        "type": "amenity"
    },
    {
        "name": "Expense Inflation",
        "value": 2,
        "type": "expense"
    },
    {
        "name": "Retail Expense Inflation",
        "value": 2,
        "type": "retail"
    }
]

retail_expenses = [
    {
        "id": "40",
        "name": "CAM",
        "statistic": None,
        "factor": "per SF",
        "cost_per": 110,
        "type": "Retail",
        "start_month": 0,
        "end_month": 0
    },
    {
        "id": "41",
        "name": "Management Fee",
        "statistic": None,
        "factor": "per SF",
        "cost_per": 10,
        "type": "Retail",
        "start_month": 0,
        "end_month": 0
    },
    {
        "id": "42",
        "name": "Insurance",
        "statistic": None,
        "factor": "per SF",
        "cost_per": 30,
        "type": "Retail",
        "start_month": 0,
        "end_month": 0
    },
    {
        "id": "43",
        "name": "Property Taxes",
        "statistic": None,
        "factor": "per SF",
        "cost_per": 0,
        "type": "Retail",
        "start_month": 0,
        "end_month": 0
    }
]

In [61]:
def get_retail_assumptions_inserts(spreadsheet, retail_income, sheet_name="Retail Assumptions", start_row=6):
    """
    Returns insert and update requests for retail income data in the Retail Assumptions sheet.
    
    Args:
        spreadsheet: The gspread spreadsheet object
        retail_income: List of retail income dictionaries
        sheet_name: Name of the sheet to update (default: "Retail Assumptions")
        start_row: Starting row for insertions (default: 6)
    
    Returns:
        tuple: (insert_request, update_payload)
    """
    ws = spreadsheet.worksheet(sheet_name)
    sheet_id = ws._properties["sheetId"]
    
    num_rows = len(retail_income)
    end_row = start_row + num_rows - 1
    
    # Insert request - insert rows starting at start_row
    insert_request = {
        "insertDimension": {
            "range": {
                "sheetId": sheet_id,
                "dimension": "ROWS",
                "startIndex": start_row - 1,  # 0-based index
                "endIndex": start_row - 1 + num_rows
            },
            "inheritFromBefore": False
        }
    }
    
    # Build rows for update - need to extend range to include all columns we're writing to
    rows = []
    for i, entry in enumerate(retail_income):
        current_row = start_row + i
        row = [""] * 145  # Columns A through EQ (145 columns total)
        
        # Column B: suite
        row[1] = entry.get("suite", "")
        
        # Column C: tenant_name  
        row[2] = entry.get("tenant_name", "")
        
        # Column D: blank
        row[3] = ""
        
        # Column E: lease_start_month
        row[4] = entry.get("lease_start_month", "")
        
        # Column F: square_feet
        row[5] = entry.get("square_feet", "")
        
        # Column G: lease_start_month (same as E)
        row[6] = entry.get("lease_start_month", "")
        
        # Column H: annual_bumps (with percent sign)
        annual_bumps = entry.get("annual_bumps", "")
        if annual_bumps != "":
            row[7] = f"{annual_bumps}%"
        else:
            row[7] = ""
        
        # Column I: rent_per_square_foot_per_year
        row[8] = entry.get("rent_per_square_foot_per_year", "")
        
        # Column J: Formula I<row> * F<row>
        row[9] = f"=I{current_row}*F{current_row}"
        
        # Columns L through EQ (columns 11 through 144): Formula pattern
        for col_idx in range(11, 145):  # L is column 11 (0-based), EQ is column 144
            # Convert column index to column letter(s)
            if col_idx < 26:
                col_letter = chr(ord('A') + col_idx)
            else:
                col_letter = chr(ord('A') + col_idx // 26 - 1) + chr(ord('A') + col_idx % 26)
            row[col_idx] = f"=($G{current_row}<={col_letter}$5)*($J{current_row}/12)*(1+$H{current_row})^(ROUNDUP(MAX({col_letter}$5-$G{current_row}+1,0)/12,0)-1)"
        
        rows.append(row)
    
    # Update payload - extend range to cover all columns we're writing to (A to EQ)
    update_payload = {
        "range": f"'{sheet_name}'!A{start_row}:EQ{end_row}",
        "values": rows
    }
    
    return insert_request, update_payload



def get_retail_assumptions_summary_row(retail_income, sheet_name='Retail Assumptions'):
    """
    Creates a summary row with formulas for the retail assumptions data.
    This row goes after all the retail income entries.
    """
    start_row = 6 + len(retail_income)  # Row after all retail entries
    end_row = start_row
    
    # Create the summary row with formulas
    summary_row = [""] * 145  # Initialize with empty strings for all columns
    
    summary_row[1] = "Total Base Retail Income"
    
    # Column E: =MIN(E6:E<6+len(retail_income))
    data_end_row = 6 + len(retail_income) - 1
    summary_row[4] = f"=MIN(E6:E{data_end_row})"
    
    # Column F: =SUM(F6:F<6+len(retail_income))
    summary_row[5] = f"=SUM(F6:F{data_end_row})"
    
    # Column G: =MIN(G6:G<6+len(retail_income))
    summary_row[6] = f"=MIN(G6:G{data_end_row})"
    
    # Column H: =IFERROR(SUMPRODUCT(H6:H<data_end>,J6:J<data_end>)/J<summary_row>,0)
    summary_row[7] = f"=IFERROR(SUMPRODUCT(H6:H{data_end_row},J6:J{data_end_row})/J{start_row},0)"
    
    # Column I: =IFERROR(SUMPRODUCT(I6:I<data_end>,F6:F<data_end>)/F<summary_row>,0)
    summary_row[8] = f"=IFERROR(SUMPRODUCT(I6:I{data_end_row},F6:F{data_end_row})/F{start_row},0)"
    
    # Column J: =SUM(J6:J<data_end>)
    summary_row[9] = f"=SUM(J6:J{data_end_row})"
    
    # Columns L through EQ (columns 11 through 144): Sum formulas
    for col_idx in range(11, 145):  # L is column 11 (0-based), EQ is column 144
        # Convert column index to column letter(s)
        if col_idx < 26:
            col_letter = chr(ord('A') + col_idx)
        else:
            col_letter = chr(ord('A') + col_idx // 26 - 1) + chr(ord('A') + col_idx % 26)
        summary_row[col_idx] = f"=SUM({col_letter}6:{col_letter}{data_end_row})"
    
    # Update payload
    update_payload = {
        "range": f"'{sheet_name}'!A{start_row}:EQ{start_row}",
        "values": [summary_row]
    }
    
    return update_payload

def get_retail_assumptions_occ_row(retail_income, sheet_name='Retail Assumptions'):
    """
    Creates an occ row below the summary row with SUMIF formulas.
    """
    start_row = 6 + len(retail_income) + 1  # Row after summary row
    data_end_row = 6 + len(retail_income) - 1  # Last data row
    summary_row_num = 6 + len(retail_income)  # Summary row number
    
    # Create the occ row with formulas
    occ_row = [""] * 145  # Initialize with empty strings for all columns
    
    occ_row[10] = "Occ."
    
    # Columns L through EO (columns 11 through 144): SUMIF formulas
    for col_idx in range(11, 145):  # L is column 11 (0-based), EO is column 144
        # Convert column index to column letter(s)
        if col_idx < 26:
            col_letter = chr(ord('A') + col_idx)
        else:
            col_letter = chr(ord('A') + col_idx // 26 - 1) + chr(ord('A') + col_idx % 26)
        occ_row[col_idx] = f"=IFERROR(SUMIF($E$6:$E${data_end_row},\"<=\"&{col_letter}5,$F$6:$F${data_end_row})/$F${summary_row_num},0)"
    
    # Update payload
    update_payload = {
        "range": f"'{sheet_name}'!A{start_row}:EQ{start_row}",
        "values": [occ_row]
    }
    
    return update_payload


def get_retail_recovery_inserts(spreadsheet, retail_income, retail_expenses, sheet_name='Retail Assumptions'):
    """
    Creates insert request and update payload for retail recovery rows.
    """
    retail_income_length = len(retail_income)
    start_row = 10 + retail_income_length  # Starting row for recovery section
    ws = spreadsheet.worksheet(sheet_name)
    sheet_id = ws._properties["sheetId"]
    # Insert request for new rows
    insert_request = {
        "insertDimension": {
            "range": {
                "sheetId": sheet_id,
                "dimension": "ROWS",
                "startIndex": start_row - 1,  # 0-based index
                "endIndex": start_row - 1 + retail_income_length
            }
        }
    }
    
    # Create data rows
    data_rows = []
    for i, item in enumerate(retail_income):
        row = [""] * 145  # Initialize with empty strings for all columns
        
        # Column B: reference to previous table (=+B6, =+B7, etc.)
        row[1] = f"=+B{6 + i}"
        
        # Column C: reference to previous table (=+C6, =+C7, etc.)
        row[2] = f"=+C{6 + i}"
        
        # Column F: recovery_start_month
        row[5] = item.get("recovery_start_month", "")
        
        # Column H: =IFERROR(F{current_row}/$F${summary_row},0)
        current_row = 10 + i + retail_income_length
        summary_row = 6 + retail_income_length  # Summary row from previous table
        row[7] = f"=IFERROR(F{current_row}/$F${summary_row},0)"
        print(current_row)

        # Column I: =IFERROR(J{current_row}/F{prev_table_row},0)
        prev_table_row = 6 + i
        row[8] = f"=IFERROR(J{current_row}/F{prev_table_row},0)"
        
        # Column J: =+H{current_row}*$J${reference_row}
        reference_row = 17 + retail_income_length * 2 + len(retail_expenses)
        row[9] = f"=+H{current_row}*$J${reference_row}"
        
        # Columns L through EO (columns 11 through 144): =IFERROR((COL$5>=$F{current_row})*$H{current_row}*COL${reference_row},0)
        for col_idx in range(11, 145):  # L is column 11 (0-based), EO is column 144
            # Convert column index to column letter(s)
            if col_idx < 26:
                col_letter = chr(ord('A') + col_idx)
            else:
                col_letter = chr(ord('A') + col_idx // 26 - 1) + chr(ord('A') + col_idx % 26)
            row[col_idx] = f"=IFERROR(({col_letter}$5>=$F{current_row})*$H{current_row}*{col_letter}${reference_row},0)"
        
        data_rows.append(row)
    
    # Update payload
    update_payload = {
        "range": f"'{sheet_name}'!A{start_row}:EQ{start_row + retail_income_length - 1}",
        "values": data_rows
    }
    
    return insert_request, update_payload


def get_retail_expenses_inserts(spreadsheet, retail_income, retail_expenses, retail_growth_rate, sheet_name):
    """
    Insert retail expenses rows and update with data
    """
    retail_income_length = len(retail_income)
    retail_expenses_length = len(retail_expenses)
    
    # Calculate starting row
    start_row = 17 + retail_income_length * 2

    ws = spreadsheet.worksheet(sheet_name)
    sheet_id = ws._properties["sheetId"]
    
    # Create insert request
    insert_request = {
        "insertDimension": {
            "range": {
                "sheetId": sheet_id,
                "dimension": "ROWS",
                "startIndex": start_row - 1,  # 0-based index
                "endIndex": start_row - 1 + retail_expenses_length
            }
        }
    }
    
    # Create data rows
    data_rows = []
    for i, expense in enumerate(retail_expenses):
        row = [""] * 145  # Initialize with empty strings for all columns
        current_row = start_row + i
        
        # Column B: expense name
        row[1] = expense.get("name", "")
        
        # Column G: retail growth rate
        row[6] = f"{retail_growth_rate}%"
        
        # Column I: cost_per
        row[8] = expense.get("cost_per", "")
        
        # Column J: =F{6 + len(retail_expenses)} * I{current_row}
        reference_row = 6 + retail_income_length
        row[9] = f"=F{reference_row}*I{current_row}"
        
        # Columns L through EO (columns 11 through 144): =IFERROR($J{current_row}/12*COL${reference_row_20}*(1+$G{current_row})^(ROUNDUP(COL$5/12,0)-1),0)
        reference_occ_row = 6 + retail_income_length + 1  
        for col_idx in range(11, 145):  # L is column 11 (0-based), EO is column 144
            # Convert column index to column letter(s)
            if col_idx < 26:
                col_letter = chr(ord('A') + col_idx)
            else:
                col_letter = chr(ord('A') + col_idx // 26 - 1) + chr(ord('A') + col_idx % 26)
            row[col_idx] = f"=IFERROR($J{current_row}/12*{col_letter}${reference_occ_row}*(1+$G{current_row})^(ROUNDUP({col_letter}$5/12,0)-1),0)"
        
        data_rows.append(row)
    
    # Update payload
    update_payload = {
        "range": f"'{sheet_name}'!A{start_row}:EO{start_row + retail_expenses_length - 1}",
        "values": data_rows
    }
    
    return insert_request, update_payload


def get_retail_expenses_summary_row(retail_income, retail_expenses, sheet_name):
    """
    Creates the summary row for retail expenses.
    Row position: 17 + len(retail_income)*2 + len(retail_expenses)
    """
    retail_income_length = len(retail_income)
    retail_expenses_length = len(retail_expenses)
    
    # Calculate row position
    summary_row = 17 + retail_income_length * 2 + retail_expenses_length
    
    # Initialize row with empty strings
    row = [""] * 145
    
    # Column B: text
    row[1] = "Total Retail Operating Expenses"
    
    # Column I: =IFERROR(J{summary_row}/F{6 + retail_income_length},0)
    reference_row = 6 + retail_income_length
    row[8] = f"=IFERROR(J{summary_row}/F{reference_row},0)"
    
    # Column J: SUM of retail expenses column J
    expenses_start_row = 17 + retail_income_length * 2
    expenses_end_row = expenses_start_row + retail_expenses_length - 1
    row[9] = f"=SUM(J{expenses_start_row}:J{expenses_end_row})"
    
    # Columns L through EO (columns 11 through 144): SUM of expense rows for that column
    for col_idx in range(11, 145):  # L is column 11 (0-based), EO is column 144
        # Convert column index to column letter(s)
        if col_idx < 26:
            col_letter = chr(ord('A') + col_idx)
        else:
            col_letter = chr(ord('A') + col_idx // 26 - 1) + chr(ord('A') + col_idx % 26)
        row[col_idx] = f"=SUM({col_letter}{expenses_start_row}:{col_letter}{expenses_end_row})"
    
    # Update payload
    update_payload = {
        "range": f"'{sheet_name}'!A{summary_row}:EO{summary_row}",
        "values": [row]
    }
    
    return update_payload



retail_growth_rate = [rate for rate in growth_rates if rate.get('type') == 'retail'][0]['value']

retail_assumptions_insert_request, retail_assumptions_update_payload = get_retail_assumptions_inserts(spreadsheet, retail_income, 'Retail Assumptions')
retail_assumptions_summary_update_payload = get_retail_assumptions_summary_row(retail_income, 'Retail Assumptions')
retail_assumptions_occ_update_payload = get_retail_assumptions_occ_row(retail_income, 'Retail Assumptions')

retail_recovery_insert_request, retail_recovery_update_payload = get_retail_recovery_inserts(spreadsheet, retail_income, retail_expenses, 'Retail Assumptions')

retail_expenses_insert_request, retail_expenses_update_payload = get_retail_expenses_inserts(spreadsheet, retail_income, retail_expenses, retail_growth_rate, 'Retail Assumptions')

retail_expenses_summary_update_payload = get_retail_expenses_summary_row(retail_income, retail_expenses, 'Retail Assumptions')

# === Run all operations ===
# spreadsheet.batch_update({"requests": [retail_assumptions_insert_request, retail_recovery_insert_request, retail_expenses_insert_request]})
spreadsheet.values_batch_update({"valueInputOption": "USER_ENTERED", "data": [retail_assumptions_update_payload, 
                                                                              retail_assumptions_summary_update_payload, 
                                                                              retail_assumptions_occ_update_payload, 
                                                                              retail_recovery_update_payload,
                                                                              retail_expenses_update_payload,
                                                                              retail_expenses_summary_update_payload]})

12
13


{'spreadsheetId': '1OikBudPoWjfOpXZXayslZAUZY7O3eM_k7UD-aGkPm9k',
 'totalUpdatedRows': 11,
 'totalUpdatedColumns': 145,
 'totalUpdatedCells': 1595,
 'totalUpdatedSheets': 1,
 'responses': [{'spreadsheetId': '1OikBudPoWjfOpXZXayslZAUZY7O3eM_k7UD-aGkPm9k',
   'updatedRange': "'Retail Assumptions'!A6:EO7",
   'updatedRows': 2,
   'updatedColumns': 145,
   'updatedCells': 290},
  {'spreadsheetId': '1OikBudPoWjfOpXZXayslZAUZY7O3eM_k7UD-aGkPm9k',
   'updatedRange': "'Retail Assumptions'!A8:EO8",
   'updatedRows': 1,
   'updatedColumns': 145,
   'updatedCells': 145},
  {'spreadsheetId': '1OikBudPoWjfOpXZXayslZAUZY7O3eM_k7UD-aGkPm9k',
   'updatedRange': "'Retail Assumptions'!A9:EO9",
   'updatedRows': 1,
   'updatedColumns': 145,
   'updatedCells': 145},
  {'spreadsheetId': '1OikBudPoWjfOpXZXayslZAUZY7O3eM_k7UD-aGkPm9k',
   'updatedRange': "'Retail Assumptions'!A12:EO13",
   'updatedRows': 2,
   'updatedColumns': 145,
   'updatedCells': 290},
  {'spreadsheetId': '1OikBudPoWjfOpXZXayslZAUZY7O3e

In [ ]:
def insert_expense_rows_to_sheet_payloads(spreadsheet, sheet_name, expenses):
    """
    Generates insert and update payloads for inserting rows into the given expense sheet.
    Returns insert_request and update_payloads instead of executing them.
    """
    ws = spreadsheet.worksheet(sheet_name)
    sheet_id = ws._properties["sheetId"]

    from string import ascii_uppercase

    def generate_excel_columns(start='J', end='EL'):
        def col_index(c):
            result = 0
            for i, char in enumerate(reversed(c)):
                result += (ascii_uppercase.index(char) + 1) * (26 ** i)
            return result

        def col_name(index):
            name = ""
            while index > 0:
                index, remainder = divmod(index - 1, 26)
                name = chr(65 + remainder) + name
            return name

        start_idx = col_index(start)
        end_idx = col_index(end)
        return [col_name(i) for i in range(start_idx, end_idx + 1)]

    month_columns = generate_excel_columns('J', 'EL')

    # Step 1: Create insert request
    insert_request = {
        "insertDimension": {
            "range": {
                "sheetId": sheet_id,
                "dimension": "ROWS",
                "startIndex": 1,
                "endIndex": 1 + len(expenses)
            },
            "inheritFromBefore": False
        }
    }

    # Step 2: Separate out "Total percent of other expenses"
    percent_expense = next((e for e in expenses if e["factor"] == "Total percent of other expenses"), None)
    normal_expenses = [e for e in expenses if e["factor"] != "Total percent of other expenses"]

    rows_to_insert = []

    # Add normal expenses first
    for i, exp in enumerate(normal_expenses):
        row_index = i + 2  # 1-based index after header row

        name = exp.get("name", "")
        factor = exp.get("factor", "")
        cost_per = float(exp.get("cost_per") or 0)
        start_month = int(exp.get("start_month") or 0)
        end_month = int(exp.get("end_month") or 0)

        # Handle statistic
        statistic = "" if factor == "Total" else float(exp.get("statistic") or 0)

        # Total formula logic
        if factor == "Total":
            total_formula = f"=C{row_index}"
        elif factor in {"per Unit", "per SF", "per Month"}:
            total_formula = f"=C{row_index}*D{row_index}"
        else:
            total_formula = ""

        row = [name, factor, cost_per, statistic, total_formula, "", start_month, end_month, ""]

        # Monthly formulas from J to EL
        for month_offset, col_letter in enumerate(month_columns):
            formula = f"=IF(AND($G{row_index}={col_letter}$1,$H{row_index}={col_letter}$1),$E{row_index},IF($G{row_index}>{col_letter}$1,0,IF($H{row_index}<{col_letter}$1,0,$E{row_index}/($H{row_index}-$G{row_index}+1))))"
            row.append(formula)

        rows_to_insert.append(row)

    # Add "Total percent of other expenses" as the last row (if it exists)
    if percent_expense:
        percent_row_index = len(rows_to_insert) + 2  # Inserted after the rest
        name = percent_expense.get("name", "")
        cost_per_val = float(percent_expense.get("cost_per") or 0)
        cost_per = f"{cost_per_val}%"  # Add percent sign

        start_month = int(percent_expense.get("start_month") or 0)
        end_month = int(percent_expense.get("end_month") or 0)

        # Statistic = sum of previous total column
        statistic_formula = f"=SUM(E2:E{percent_row_index - 1})"
        total_formula = f"=C{percent_row_index}*D{percent_row_index}"

        row = [name, "Total percent of other expenses", cost_per, statistic_formula, total_formula, "", start_month, end_month, ""]

        for col_letter in month_columns:
            formula = f"=IF(AND($G{percent_row_index}={col_letter}$1,$H{percent_row_index}={col_letter}$1),$E{percent_row_index},IF($G{percent_row_index}>{col_letter}$1,0,IF($H{percent_row_index}<{col_letter}$1,0,$E{percent_row_index}/($H{percent_row_index}-$G{percent_row_index}+1))))"
            row.append(formula)

        rows_to_insert.append(row)

    # Step 3: Create main data update payload
    start_row = 2
    end_row = 1 + len(rows_to_insert)
    end_col_letter = "EL"
    range_str = f"'{sheet_name}'!A{start_row}:{end_col_letter}{end_row}"
    
    main_update_payload = {
        "range": range_str,
        "values": rows_to_insert
    }

    # Step 4: Create Total row update payloads
    total_row_index = end_row + 1

    # Update E (total column)
    total_formula_value = f"=SUM(E2:E{end_row})"
    total_column_update_payload = {
        "range": f"{sheet_name}!E{total_row_index}",
        "values": [[total_formula_value]]
    }

    # Update each month column (J to EL) with a SUM formula
    month_sums = []
    for col_letter in month_columns:
        formula = f"=SUM({col_letter}2:{col_letter}{end_row})"
        month_sums.append(formula)

    # Create month totals update payload
    month_totals_update_payload = {
        "range": f"{sheet_name}!J{total_row_index}:{month_columns[-1]}{total_row_index}",
        "values": [month_sums]
    }

    return insert_request, [main_update_payload, total_column_update_payload, month_totals_update_payload]

closing_costs_filtered = [expense for expense in expenses_json if expense.get('type') == 'Closing Costs']
closing_cost_insert_request, closing_cost_update_payloads = insert_expense_rows_to_sheet_payloads(spreadsheet, 'Closing Costs', closing_costs_filtered)
hard_costs_filtered = [expense for expense in expenses_json if expense.get('type') == 'Hard Costs']
hard_costs_insert_request, hard_costs_update_payloads = insert_expense_rows_to_sheet_payloads(spreadsheet, 'Hard Costs', hard_costs_filtered)
legal_costs_filtered = [expense for expense in expenses_json if expense.get('type') == 'Legal and Pre-Development Costs']
legal_costs_insert_request, legal_costs_update_payloads = insert_expense_rows_to_sheet_payloads(spreadsheet, 'Legal and Pre-Development Costs', legal_costs_filtered)
reserves_filtered = [expense for expense in expenses_json if expense.get('type') == 'Reserves']
reserves_insert_request, reserves_update_payloads = insert_expense_rows_to_sheet_payloads(spreadsheet, 'Reserves', reserves_filtered)


In [ ]:
retail_growth_rates